In [1]:
import music21
from music21 import converter as m21_converter

In [ ]:
from pathlib import Path
from music21.musicxml.m21ToXml import MusicXMLExportException
import math

a = m21_converter.parse(r'tests\\data\\ActorPreludeSample.musicxml')

# Normalize notation to help ensure durations are expressible for MusicXML export
try:
    a.makeNotation(inPlace=True)
except Exception as e:
    print(f'makeNotation warning: {e}')

def try_write(score, fp):
    try:
        # Force makeNotation during export as well, per music21 guidance
        return score.write('musicxml', fp=fp, makeNotation=True)
    except MusicXMLExportException as mxe:
        print(f'Write failed with MusicXMLExportException: {mxe}')
        return None
    except Exception as e:
        print(f'Write failed: {e}')
        return None

out_path = 'ActorPreludeSample_out.musicxml'
out_file = try_write(a, out_path)

if out_file is None:
    # Fallback 1: split at common durational boundaries to avoid inexpressible durations
    try:
        print("Using splitAtDurations fallback...")
        a = a.splitAtDurations([0.015625,0.03125,0.0625,0.125, 0.25, 0.5, 1.0, 2.0, 4.0])
        a.makeNotation(inPlace=True)
    except Exception as e:
        print(f'splitAtDurations warning: {e}')
    out_file = try_write(a, out_path)

if out_file is None:
    # Fallback 2: round all durations to a 1/96 grid to avoid float artifacts
    try:
        for el in a.recurse().notesAndRests:
            ql = el.duration.quarterLength
            if ql is not None:
                el.duration.quarterLength = round(ql * 96) / 96.0
        a.makeNotation(inPlace=True)
    except Exception as e:
        print(f'Quantize-to-grid warning: {e}')
    out_file = try_write(a, out_path)

if out_file:
    print(f'Wrote MusicXML to: {Path(out_file).resolve()}')
else:
    raise RuntimeError('Failed to export a valid MusicXML file after normalization attempts.')

Write failed with MusicXMLExportException: In part (Timpani), measure (27): Cannot convert inexpressible durations to MusicXML.
splitAtDurations warning: Stream.splitAtDurations() takes 1 positional argument but 2 were given
Write failed with MusicXMLExportException: In part (Timpani), measure (27): Cannot convert inexpressible durations to MusicXML.
Write failed with MusicXMLExportException: In part (Timpani), measure (27): Cannot convert inexpressible durations to MusicXML.


musicxml.m21ToXml: WARNING: we are out of midi channels! help!
musicxml.m21ToXml: WARNING: we are out of midi channels! help!
musicxml.m21ToXml: WARNING: we are out of midi channels! help!
musicxml.m21ToXml: WARNING: we are out of midi channels! help!


Wrote MusicXML to: C:\Users\DMATTOS\notare\ActorPreludeSample_out.musicxml


musicxml.m21ToXml: WARNING: we are out of midi channels! help!
musicxml.m21ToXml: WARNING: we are out of midi channels! help!
